In [4]:
# importing the libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# defining the parameters

root_dir = os.path.dirname(os.path.abspath(os.getcwd()))
pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", None)
plt.rcParams["figure.figsize"] = (12, 8)
sns.set_style("darkgrid")

In [6]:
# defining the paths
trainpath = os.path.join(root_dir, "data", "train.csv")
item_data_path = os.path.join(root_dir, "data", "item_data.csv")
view_log_path = os.path.join(root_dir, "data", "view_log.csv")
testpath = os.path.join(root_dir, "data", "test.csv")

# importing the datasets
traindf = pd.read_csv(trainpath)
item_data_df = pd.read_csv(item_data_path)
view_log_df = pd.read_csv(view_log_path)
testdf = pd.read_csv(testpath)

In [7]:
traindf.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


### Column descriptions - 
- impression_id = Ad impression ID 1 (unique). Shouldn't be fed to the model
- impression_time = time of impression at the partner site
- user_id = user identification
- app code = Application Code for a partner website where the ad was shown
- os_version = version of operating system
- is_4G = 1 (Using 4G), 0 (No 4G)
- is_click = **target**; 0 (No Click), 1 (Click)

In [8]:
item_data_df.head()

,item_id,item_price,category_1,category_2,category_3,product_type
0,26880,4602,11,35,20,3040
1,54939,3513,12,57,85,6822
2,40383,825,17,8,279,1619
3,8777,2355,13,58,189,5264
4,113705,1267,17,39,151,10239


Variables are quite self-explanatory. But can't be mapped to the main column as there is not shared column. 
- item_id is unique in this dataset

In [9]:
view_log_df.head()

,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333,4557,32970
1,2018-10-15 08:58:00,android,503590,74788,7640
2,2018-10-15 08:58:00,android,573960,23628,128855
3,2018-10-15 08:58:00,android,121691,2430,12774
4,2018-10-15 08:58:00,android,218564,19227,28296


Column Description-
- server_time = Timestamp of the log
- device_type = Device type of the user
- browser_id = Browser session id
- user_id = can be used to map this table to traindf
- item_id = can be used to map this table to item_data_df

In [10]:
# shape of all the datasets
print("Shape of traindf = {}".format(traindf.shape))
print("Shape of testdf = {}".format(testdf.shape))
print("Shape of item_data_df = {}".format(item_data_df.shape))
print("Shape of view_log_df = {}".format(view_log_df.shape))

Shape of traindf = (237609, 7)
Shape of testdf = (90675, 6)
Shape of item_data_df = (132761, 6)
Shape of view_log_df = (3118622, 5)


In [11]:
def info_df(df):
    """
    creates a dataframe that returns the number of uniques and nulls for each column
    
    input: dataframe
    output: dataframe of shape 2*number of columns
    """
    return pd.DataFrame({
        "uniques": df.nunique(),
        "nulls": df.isnull().sum()
    }).T

In [12]:
info_df(traindf)

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
uniques,237609,36461,74723,490,3,2,2
nulls,0,0,0,0,0,0,0


In [13]:
info_df(item_data_df)

,item_id,item_price,category_1,category_2,category_3,product_type
uniques,132761,13531,17,79,335,7959
nulls,0,0,0,0,0,0


In [14]:
info_df(view_log_df)

,server_time,device_type,session_id,user_id,item_id
uniques,82840,3,1014970,89157,126708
nulls,0,0,0,0,0


In [15]:
# merging view_log_df and item_data_df
supplement_df = pd.merge(view_log_df, item_data_df, how = "inner", on = "item_id")

print("shape of supplement_df = {}".format(supplement_df.shape))

shape of supplement_df = (3116840, 10)


The len of ``supplement_df`` is smaller than the ``view_log_df``, which signify the not all the ``item_id``s are present in the ``item_data_df`` that are there in ``view_log_df``.

In [16]:
# checking for the item_ids that are in view_log_df but not in item_data_df
#unfound_items = []
#for i in list(view_log_df.item_id.unique()):
#    if i not in list(item_data_df.item_id.unique()):
#        unfound_items.append(i)


#when we ran above code, it is found that around 101 items_ids are not present in item_data_df

In [17]:
supplement_df.head()

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
0,2018-10-15 08:58:00,android,112333,4557,32970,54685,16,56,253,3184
1,2018-10-15 09:36:00,android,783457,88320,32970,54685,16,56,253,3184
2,2018-10-15 10:59:00,android,6902,1711,32970,54685,16,56,253,3184
3,2018-10-15 11:31:00,android,61138,58906,32970,54685,16,56,253,3184
4,2018-10-15 12:03:00,android,441653,64221,32970,54685,16,56,253,3184


In [18]:
info_df(supplement_df)

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
uniques,82840,3,1014725,89152,126607,13207,17,79,335,7846
nulls,0,0,0,0,0,0,0,0,0,0


In [19]:
# changing the server time to datetime format
supplement_df["server_time"] = pd.to_datetime(supplement_df["server_time"])

### Aggregating the ``supplement_df`` on ``user id``

In [20]:
# converting date to timestamp
supplement_df.loc[:, "server_time"] = pd.DatetimeIndex(supplement_df["server_time"]).astype(np.int64) * 1e-9

# label encoding device type
supplement_df["device_type"] = supplement_df["device_type"].map({
    "android": 0,
    "iphone": 1,
    "web": 2
})

In [21]:
# defining aggregate functions
agg_func = {
    "server_time": [np.ptp],
    "session_id": ["nunique"],
    "device_type": ["mean", "nunique"],
    "item_price": ["sum", "mean", "max", "min", "std"],
    "category_1": ["sum", "mean", "max", "min", "std"],
    "category_2": ["sum", "mean", "max", "min", "std"],
    "category_3": ["sum", "mean", "max", "min", "std"],
    "product_type": ["sum", "mean", "max", "min", "std"]
}

# aggregating the supplement_df
agg_logs = supplement_df.groupby(["user_id"]).agg(agg_func)

/home/arjun/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [22]:
# define prefix to discriminate b/w column names
prefix = "vl_"

# strip removes the leading and trailing characters mentioned. If no argument is passed, whitespaces are removed
agg_logs.columns = [prefix + "_".join(col).strip() for col in agg_logs.columns.values]
    
# setting the index
agg_logs.reset_index(inplace = True)
    
temp_df = supplement_df.groupby("user_id").size().reset_index(name = "{}logs".format(prefix))

agg_logs = pd.merge(temp_df, agg_logs, on = "user_id", how = "left")

In [23]:
agg_logs.head()

,user_id,vl_logs,vl_server_time_ptp,vl_device_type_mean,vl_item_price_sum,vl_item_price_mean,vl_item_price_max,vl_item_price_min,vl_item_price_std,vl_category_1_sum,vl_category_1_mean,vl_category_1_max,vl_category_1_min,vl_category_1_std,vl_category_2_sum,vl_category_2_mean,vl_category_2_max,vl_category_2_min,vl_category_2_std,vl_category_3_sum,vl_category_3_mean,vl_category_3_max,vl_category_3_min,vl_category_3_std,vl_product_type_sum,vl_product_type_mean,vl_product_type_max,vl_product_type_min,vl_product_type_std
0,0,42,4341900.0,0.0,394618,9395.666667,92160,332,21378.979860,415,9.880952,17,1,5.751786,1521,36.214286,79,7,18.612264,7206,171.571429,308,30,72.080288,252817,6019.452381,10457,60,3529.301383
1,1,8,2996580.0,0.0,31574,3946.750000,12595,383,4412.414232,77,9.625000,15,0,4.657943,357,44.625000,78,0,31.995256,1147,143.375000,277,44,73.911023,50643,6330.375000,10112,3303,2214.511422
2,2,165,4697280.0,0.0,2443569,14809.509091,281536,128,31645.142268,1610,9.757576,17,0,5.504047,6213,37.654545,78,1,20.912318,30336,183.854545,336,4,95.397963,939681,5695.036364,10561,20,3078.440597
3,3,8,1740.0,0.0,58059,7257.375000,16640,537,7805.928863,45,5.625000,14,0,6.947507,324,40.500000,62,6,23.219450,907,113.375000,157,50,44.316838,31060,3882.500000,6659,491,2911.447504
4,4,2,60.0,0.0,60229,30114.500000,58252,1977,39792.434111,23,11.500000,14,9,3.535534,64,32.000000,61,3,41.012193,186,93.000000,159,27,93.338095,4660,2330.000000,4426,234,2964.191627


In [24]:
print("Shape of traindf = {}".format(traindf.shape))
print("Shape of testdf = {}".format(testdf.shape))

trainset = pd.merge(traindf, agg_logs, on = "user_id", how = "left")
testset = pd.merge(testdf, agg_logs, on = "user_id", how = "left")

print("Shape of traindf = {}".format(trainset.shape))
print("Shape of testdf = {}".format(testset.shape))

Shape of traindf = (237609, 7)
Shape of testdf = (90675, 6)
Shape of traindf = (237609, 35)
Shape of testdf = (90675, 34)


In [25]:
trainset.to_csv(os.path.join(root_dir, "data", "aggregated_train.csv"), index = False)
testset.to_csv(os.path.join(root_dir, "data", "aggregated_test.csv"), index = False)